In [ ]:
# pip install seaborn

Note: you may need to restart the kernel to use updated packages.


In [3]:
import pyodbc
import pandas as pd
from azure.identity import DefaultAzureCredential
from azure.keyvault.secrets import SecretClient
from datetime import datetime, timedelta

import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [4]:
def get_secret():
    credential = DefaultAzureCredential()
    secret_client = SecretClient(vault_url="https://kvsynapsedata001.vault.azure.net", credential=credential)
    secret = secret_client.get_secret("pw-SRVSQLPoolSQL")
    return secret.value
 
 
server = 'tcp:arriba-synapseworkspace-prod-ae-001.sql.azuresynapse.net,1433'
database = 'arribasqlpool1'
username = 'SRV_SQLPool_SQL'
password = get_secret()  
 
otheropts = 'Encrypt=yes;TrustServerCertificate=yes;Connection Timeout=30;'
 
 
cnxn_t = pyodbc.connect('DRIVER={ODBC Driver 18 for SQL Server};SERVER='+server+';DATABASE='+database+';UID='+username+';PWD='+password+';'+otheropts)
cursor_t = cnxn_t.cursor()
 
 
 
 

##### Sample Queries

##### AB tables from JRL

In [14]:
query_rm_rev = """SELECT * FROM [vinci].[tBI_RM_Revenue]"""
rm_rev_df = pd.read_sql(query_rm_rev, cnxn_t)

# ab_query_type_df = pd.read_sql("""SELECT * FROM [aimbig].[query_types]""", cnxn_t)

/tmp/ipykernel_46461/782974869.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  rm_rev_df = pd.read_sql(query_rm_rev, cnxn_t)


In [15]:
rm_rev_df.head()

,CaseBillId,BillDate,BillerId,FirstName,LastName,AmountExclGST,GST,AmountInclGST,Units,Qty,ActivityName,TeamName,AccountingLocation,EmployeeNo,AreaName
0,14254037,2024-09-03,8511,Alexia,Armiento,3354,335,3689,1.5,0.15,Communication with Stakeholder - to / from WAS...,South Australia,9,80321,SA
1,14749498,2025-03-21,8511,Alexia,Armiento,4182,418,4600,2.0,0.20,Communication - Worker (Email),South Australia,9,80321,SA
2,14732557,2025-03-14,11297,Martin,Wu,6242,624,6866,3.0,0.30,Communication,South Coast,4,80425,NSW South
3,15025814,2025-07-04,8478,Ayden,Demeijer,21300,2130,23430,10.0,1.00,Initial Assessment Report,North TAS,23,80183,TAS
4,14278688,2024-09-11,8511,Alexia,Armiento,25048,2505,27553,12.0,1.20,Communication - Injured Worker,South Australia,9,80321,SA


In [16]:
from datetime import datetime
from zoneinfo import ZoneInfo

sydney_time = datetime.now(ZoneInfo("Australia/Sydney"))

In [17]:
# 构建文件名
filename = f"today_closed_record_{sydney_time}.parquet"

# 保存 DataFrame
rm_rev_df.to_parquet(filename, index=False)

#### Start Accessing parquet static parquet file

In [59]:
# df = pd.read_parquet('today_closed_record_2025-07-24 10:55:29.353587+10:00.parquet')
df = pd.read_parquet('today_closed_record_2025-07-24 11:01:57.927579+10:00.parquet')

In [60]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1004270 entries, 0 to 1004269
Data columns (total 15 columns):
 #   Column              Non-Null Count    Dtype  
---  ------              --------------    -----  
 0   CaseBillId          1004270 non-null  int64  
 1   BillDate            1004270 non-null  object 
 2   BillerId            1004270 non-null  int64  
 3   FirstName           1004270 non-null  object 
 4   LastName            1004270 non-null  object 
 5   AmountExclGST       1004270 non-null  int64  
 6   GST                 1004270 non-null  int64  
 7   AmountInclGST       1004270 non-null  int64  
 8   Units               1004270 non-null  float64
 9   Qty                 1004270 non-null  float64
 10  ActivityName        1004270 non-null  object 
 11  TeamName            1004270 non-null  object 
 12  AccountingLocation  1004270 non-null  object 
 13  EmployeeNo          1004270 non-null  object 
 14  AreaName            1004270 non-null  object 
dtypes: float64(2), 

In [61]:
df.head()

,CaseBillId,BillDate,BillerId,FirstName,LastName,AmountExclGST,GST,AmountInclGST,Units,Qty,ActivityName,TeamName,AccountingLocation,EmployeeNo,AreaName
0,14254037,2024-09-03,8511,Alexia,Armiento,3354,335,3689,1.5,0.15,Communication with Stakeholder - to / from WAS...,South Australia,9,80321,SA
1,14749498,2025-03-21,8511,Alexia,Armiento,4182,418,4600,2.0,0.20,Communication - Worker (Email),South Australia,9,80321,SA
2,14732557,2025-03-14,11297,Martin,Wu,6242,624,6866,3.0,0.30,Communication,South Coast,4,80425,NSW South
3,15025814,2025-07-04,8478,Ayden,Demeijer,21300,2130,23430,10.0,1.00,Initial Assessment Report,North TAS,23,80183,TAS
4,14278688,2024-09-11,8511,Alexia,Armiento,25048,2505,27553,12.0,1.20,Communication - Injured Worker,South Australia,9,80321,SA


In [62]:

df['BillDate'] = pd.to_datetime(df['BillDate'])

# Filter for July 2025
july_df = df[df['BillDate'].dt.to_period('M') == '2025-07']

# Group by Team Name and sum the amounts
summary_df = july_df.groupby('TeamName')[['AmountInclGST', 'AmountExclGST']].sum().reset_index()


In [63]:
summary_df


,TeamName,AmountInclGST,AmountExclGST
0,Account Management,220000,200000
1,Bendigo & Mildura,4975819,4523496
2,Canberra,22000604,20000602
3,Client Relations Managers,1018850,926241
4,Frankston,10712539,9738729
5,Geelong & Ballarat,8858810,8054593
6,Gold Coast,12505570,11368707
7,Liverpool & Campbelltown,30368185,27607199
8,Maitland,9378945,8526273
9,Moonee Ponds,25274991,22977360
